# Logistic Regression Example Code Along

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("code_along").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [7]:
my_data.show()
# Notice that the data is already nicely formatted.
# Label is in binary format:  0 OR 1.

my_data.head(1)[0]  # Notice that we get back a SparseVector.

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0,

**Create the Logistic Regression Model.**

In [8]:
# Create an instance of the LogisticRegression model.
log_reg_model = LogisticRegression()

In [9]:
fitted_log_reg_model = log_reg_model.fit(my_data)

**Now get a summary of our model.**

In [10]:
log_summary = fitted_log_reg_model.summary

In [13]:
# Grab the predictions DataFrame:
log_summary.predictions.printSchema()
log_summary.predictions.show()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...

**Let us now introduce the concept of evaluators.**

In [14]:
train_data, test_data = my_data.randomSplit([0.7, 0.3], seed=123)
# We are going to retrain on train_data.
# Then we will evaluate on test_data.

In [15]:
# Instantiate a LogisticRegression model.
log_reg_fin = LogisticRegression()

In [16]:
# Fit the model only on the training data.
fitted_log_reg_fin = log_reg_fin.fit(train_data)

In [17]:
# Evaluate the model on a test dataset. 
predictions_and_labels = fitted_log_reg_fin.evaluate(test_data)

In [21]:
predictions_and_labels
# <pyspark.ml.classification.BinaryLogisticRegressionSummary at 0x7f735670e588>

In [22]:
predictions_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[23.2695668264462...|[0.99999999992162...|       0.0|
|  0.0|(692,[100,101,102...|[6.74341003223320...|[0.99882276635250...|       0.0|
|  0.0|(692,[121,122,123...|[21.5462662869559...|[0.99999999956088...|       0.0|
|  0.0|(692,[123,124,125...|[20.3443168734083...|[0.99999999853925...|       0.0|
|  0.0|(692,[124,125,126...|[31.0536624793439...|[0.99999999999996...|       0.0|
|  0.0|(692,[125,126,127...|[24.3839045095629...|[0.99999999997428...|       0.0|
|  0.0|(692,[126,127,128...|[25.2122648648335...|[0.99999999998876...|       0.0|
|  0.0|(692,[126,127,128...|[22.6769435308488...|[0.99999999985824...|       0.0|
|  0.0|(692,[127,128,129...|[20.9007588383881...|[0.99999999916263...|       0.0|
|  0.0|(692,[128

**Let us further explore the ```BinaryClassificationEvaluator.```**

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator

In [24]:
my_eval = BinaryClassificationEvaluator()  # Note that we keep all the defaults.

In [25]:
my_final_roc = my_eval.evaluate(predictions_and_labels.predictions)

In [27]:
my_final_roc
# 1.0 --> Essentially means that we achieved a perfect fit.
# Very idealistic.  Not likely to happen in real-world applications.
# Not a realistic example.

1.0